<a href="https://colab.research.google.com/github/Ujjwalb2/Rnn-model-chatbot/blob/main/chatbot_using_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import pickle
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import stopwords
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from io import StringIO
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score



In [ ]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
file=open('//content//train_qa.txt','rb')
data1=pickle.load(file)

In [ ]:
df2 = pd.DataFrame(columns=['Context', 'Question', 'Answer'])
for data in data1:
    df2 = df2.append({'Context': ' '.join(data[0]), 'Question': ' '.join(data[1]), 'Answer': data[2]}, ignore_index=True)


In [ ]:
df=df2

In [ ]:
df

,Context,Question,Answer
0,Mary moved to the bathroom . Sandra journeyed ...,Is Sandra in the hallway ?,no
1,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bathroom ?,no
2,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the office ?,no
3,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes
4,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes
...,...,...,...
9995,Daniel went to the office . Sandra journeyed t...,Is Sandra in the kitchen ?,no
9996,Daniel went to the office . Sandra journeyed t...,Is Daniel in the bedroom ?,yes
9997,Daniel went to the office . Sandra journeyed t...,Is Sandra in the hallway ?,no
9998,Daniel went to the office . Sandra journeyed t...,Is Mary in the kitchen ?,no


In [ ]:
def data_processing_1(data):
  df['Context'].dropna(inplace=True)
  df['Context'] = [entry.lower() for entry in df['Context']]
  df['Context']= [word_tokenize(entry) for entry in df['Context']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(df['Context']):
    # print(index, entry)
    # print("--------------------------------")
    Final_words = []
    for word, tag in pos_tag(entry):
        # print(word, tag)
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,pos = tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index,'Context_final'] = str(Final_words)


  return data

In [ ]:
def data_processing_2(data):
  df['Question'].dropna(inplace=True)
  df['Question'] = [entry.lower() for entry in df['Question']]
  df['Quesiton']= [word_tokenize(entry) for entry in df['Question']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(df['Question']):
    Final_words_2 = []
    for word, tag in pos_tag(entry.split()):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final_2 = word_Lemmatized.lemmatize(word, pos=tag_map.get(tag[0], 'n'))
            Final_words_2.append(word_Final_2)
    df.loc[index, 'Question_final'] = str(Final_words_2)

  return data




In [ ]:
def data_processing_3(data):
  df['Answer'].dropna(inplace=True)
  df['Answer'] = [entry.lower() for entry in df['Answer']]
  df['Answer']= [word_tokenize(entry) for entry in df['Answer']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(df['Answer']):
    Final_words_2 = []
    for word, tag in pos_tag(entry):
        if  word.isalpha():
            word_Final_2 = word_Lemmatized.lemmatize(word, pos=tag_map.get(tag[0], 'n'))
            Final_words_2.append(word_Final_2)
    df.loc[index, 'Answer_final'] = str(Final_words_2)

  return data

In [ ]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [ ]:
df=data_processing_1(df)


In [ ]:
df=data_processing_2(df)

In [ ]:
df=data_processing_3(df)

In [ ]:
df

,Context,Question,Answer,Context_final,Quesiton,Question_final,Answer_final
0,"[mary, moved, to, the, bathroom, ., sandra, jo...",is sandra in the hallway ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
1,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bathroom ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bathroom, ?]","['daniel', 'bathroom']",['no']
2,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the office ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, office, ?]","['daniel', 'office']",['no']
3,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bedroom ?,[yes],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
4,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bedroom ?,[yes],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
...,...,...,...,...,...,...,...
9995,"[daniel, went, to, the, office, ., sandra, jou...",is sandra in the kitchen ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, kitchen, ?]","['sandra', 'kitchen']",['no']
9996,"[daniel, went, to, the, office, ., sandra, jou...",is daniel in the bedroom ?,[yes],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
9997,"[daniel, went, to, the, office, ., sandra, jou...",is sandra in the hallway ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
9998,"[daniel, went, to, the, office, ., sandra, jou...",is mary in the kitchen ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, mary, in, the, kitchen, ?]","['mary', 'kitchen']",['no']


In [ ]:
df.drop('Context', axis=1, inplace=True)
df.drop('Answer', axis=1, inplace=True)



In [ ]:
df.drop('Quesiton', axis=1, inplace=True)

In [ ]:
df.drop('Question', axis=1, inplace=True)

In [ ]:
df

,Context_final,Question_final,Answer_final
0,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['sandra', 'hallway']",['no']
1,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'bathroom']",['no']
2,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'office']",['no']
3,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'bedroom']",['yes']
4,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'bedroom']",['yes']
...,...,...,...
9995,"['daniel', 'go', 'office', 'sandra', 'journeye...","['sandra', 'kitchen']",['no']
9996,"['daniel', 'go', 'office', 'sandra', 'journeye...","['daniel', 'bedroom']",['yes']
9997,"['daniel', 'go', 'office', 'sandra', 'journeye...","['sandra', 'hallway']",['no']
9998,"['daniel', 'go', 'office', 'sandra', 'journeye...","['mary', 'kitchen']",['no']


In [ ]:
y=df['Answer_final']
x=df.drop('Answer_final', axis=1)

In [ ]:
Encoder = LabelEncoder()
Train_y = Encoder.fit_transform(y)


In [ ]:
# Tfidf_vect = TfidfVectorizer(max_features=5000)
# Tfidf_vect.fit(df['Context_final'])
# Tfidf_vect.fit(df['Question_final'])

In [ ]:
# Train_X_Tfidf = Tfidf_vect.transform(x)


In [ ]:
Train_y

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
x

,Context_final,Question_final
0,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['sandra', 'hallway']"
1,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'bathroom']"
2,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'office']"
3,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'bedroom']"
4,"['mary', 'move', 'bathroom', 'sandra', 'journe...","['daniel', 'bedroom']"
...,...,...
9995,"['daniel', 'go', 'office', 'sandra', 'journeye...","['sandra', 'kitchen']"
9996,"['daniel', 'go', 'office', 'sandra', 'journeye...","['daniel', 'bedroom']"
9997,"['daniel', 'go', 'office', 'sandra', 'journeye...","['sandra', 'hallway']"
9998,"['daniel', 'go', 'office', 'sandra', 'journeye...","['mary', 'kitchen']"


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:

# Tokenize the words in the 'Context_final' and 'Question_final' columns
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x['Context_final'] + x['Question_final'])

# Convert words to sequences of integers
X_sequences_context = tokenizer.texts_to_sequences(x['Context_final'])
X_sequences_question = tokenizer.texts_to_sequences(x['Question_final'])

# Pad sequences to make them of equal length
X_padded_context = pad_sequences(X_sequences_context)
X_padded_question = pad_sequences(X_sequences_question)

# Concatenate the padded sequences
X_combined_padded = np.concatenate((X_padded_context, X_padded_question), axis=1)

# Reshape X to have 3D shape (number of samples, time steps, number of features)
X_combined_padded = X_combined_padded.reshape(X_combined_padded.shape[0], 1, X_combined_padded.shape[1])

# Encode the target variable if it's not in numerical form
encoder_context = LabelEncoder()
y_encoded = encoder_context.fit_transform(Train_y)

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(X_combined_padded.shape[1], X_combined_padded.shape[2])))

model.add(Dense(len(encoder_context.classes_), activation='softmax'))  # Adjust activation based on your task

# Use 'sparse_categorical_crossentropy' as the loss function for multiclass classification
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model on the entire dataset
model.fit(X_combined_padded, Train_y, epochs=75, batch_size=32)


Epoch 1/75
313/313 [==============================] - 4s 3ms/step - loss: 0.7011 - accuracy: 0.5087
Epoch 2/75
313/313 [==============================] - 1s 4ms/step - loss: 0.6936 - accuracy: 0.5093
Epoch 3/75
313/313 [==============================] - 1s 4ms/step - loss: 0.6909 - accuracy: 0.5307
Epoch 4/75
313/313 [==============================] - 1s 2ms/step - loss: 0.6902 - accuracy: 0.5310
Epoch 5/75
313/313 [==============================] - 1s 2ms/step - loss: 0.6886 - accuracy: 0.5289
Epoch 6/75
313/313 [==============================] - 1s 2ms/step - loss: 0.6876 - accuracy: 0.5307
Epoch 7/75
313/313 [==============================] - 1s 3ms/step - loss: 0.6855 - accuracy: 0.5460
Epoch 8/75
313/313 [==============================] - 1s 2ms/step - loss: 0.6835 - accuracy: 0.5549
Epoch 9/75
313/313 [==============================] - 1s 2ms/step - loss: 0.6820 - accuracy: 0.5499
Epoch 10/75
313/313 [==============================] - 1s 2ms/step - loss: 0.6808 - accuracy: 0.5599

# **predicting**
# New Section

In [ ]:
file1=open('/content/test_qa.txt','rb')
data2=pickle.load(file1)

In [ ]:
df3 = pd.DataFrame(columns=['Context', 'Question', 'Answer'])
for data in data1:
    df3 = df3.append({'Context': ' '.join(data[0]), 'Question': ' '.join(data[1]), 'Answer': data[2]}, ignore_index=True)


Streaming output truncated to the last 5000 lines.
<ipython-input-31-197de5ec618c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append({'Context': ' '.join(data[0]), 'Question': ' '.join(data[1]), 'Answer': data[2]}, ignore_index=True)
<ipython-input-31-197de5ec618c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append({'Context': ' '.join(data[0]), 'Question': ' '.join(data[1]), 'Answer': data[2]}, ignore_index=True)
<ipython-input-31-197de5ec618c>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df3.append({'Context': ' '.join(data[0]), 'Question': ' '.join(data[1]), 'Answer': data[2]}, ignore_index=True)
<ipython-input-31-197de5ec618c>:3: FutureWarning: The frame.append method is deprecat

In [ ]:
df3

,Context,Question,Answer
0,Mary moved to the bathroom . Sandra journeyed ...,Is Sandra in the hallway ?,no
1,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bathroom ?,no
2,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the office ?,no
3,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes
4,Mary moved to the bathroom . Sandra journeyed ...,Is Daniel in the bedroom ?,yes
...,...,...,...
9995,Daniel went to the office . Sandra journeyed t...,Is Sandra in the kitchen ?,no
9996,Daniel went to the office . Sandra journeyed t...,Is Daniel in the bedroom ?,yes
9997,Daniel went to the office . Sandra journeyed t...,Is Sandra in the hallway ?,no
9998,Daniel went to the office . Sandra journeyed t...,Is Mary in the kitchen ?,no


In [ ]:
def data_processing_12(data):
  df3['Context'].dropna(inplace=True)
  df3['Context'] = [entry.lower() for entry in df3['Context']]
  df3['Context']= [word_tokenize(entry) for entry in df3['Context']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(df3['Context']):

    Final_words = []
    for word, tag in pos_tag(entry):
        # print(word, tag)
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,pos = tag_map[tag[0]])
            Final_words.append(word_Final)
    df3.loc[index,'Context_final'] = str(Final_words)


  return data

In [ ]:
def data_processing_22(data):
  df3['Question'].dropna(inplace=True)
  df3['Question'] = [entry.lower() for entry in df3['Question']]
  df3['Quesiton']= [word_tokenize(entry) for entry in df3['Question']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(df3['Question']):
    Final_words_2 = []
    for word, tag in pos_tag(entry.split()):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final_2 = word_Lemmatized.lemmatize(word, pos=tag_map.get(tag[0], 'n'))
            Final_words_2.append(word_Final_2)
    df3.loc[index, 'Question_final'] = str(Final_words_2)

  return data




In [ ]:
def data_processing_32(data):
  df3['Answer'].dropna(inplace=True)
  df3['Answer'] = [entry.lower() for entry in df3['Answer']]
  df3['Answer']= [word_tokenize(entry) for entry in df3['Answer']]
  word_Lemmatized = WordNetLemmatizer()
  for index, entry in enumerate(df3['Answer']):
    Final_words_2 = []
    for word, tag in pos_tag(entry):
        if  word.isalpha():
            word_Final_2 = word_Lemmatized.lemmatize(word, pos=tag_map.get(tag[0], 'n'))
            Final_words_2.append(word_Final_2)
    df3.loc[index, 'Answer_final'] = str(Final_words_2)

  return data

In [ ]:
df3=data_processing_12(df3)
df3=data_processing_22(df3)
df3=data_processing_32(df3)

In [ ]:
df3

,Context,Question,Answer,Context_final,Quesiton,Question_final,Answer_final
0,"[mary, moved, to, the, bathroom, ., sandra, jo...",is sandra in the hallway ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
1,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bathroom ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bathroom, ?]","['daniel', 'bathroom']",['no']
2,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the office ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, office, ?]","['daniel', 'office']",['no']
3,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bedroom ?,[yes],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
4,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bedroom ?,[yes],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
...,...,...,...,...,...,...,...
9995,"[daniel, went, to, the, office, ., sandra, jou...",is sandra in the kitchen ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, kitchen, ?]","['sandra', 'kitchen']",['no']
9996,"[daniel, went, to, the, office, ., sandra, jou...",is daniel in the bedroom ?,[yes],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
9997,"[daniel, went, to, the, office, ., sandra, jou...",is sandra in the hallway ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
9998,"[daniel, went, to, the, office, ., sandra, jou...",is mary in the kitchen ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, mary, in, the, kitchen, ?]","['mary', 'kitchen']",['no']


In [ ]:
df3

,Context,Question,Answer,Context_final,Quesiton,Question_final,Answer_final
0,"[mary, moved, to, the, bathroom, ., sandra, jo...",is sandra in the hallway ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
1,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bathroom ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bathroom, ?]","['daniel', 'bathroom']",['no']
2,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the office ?,[no],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, office, ?]","['daniel', 'office']",['no']
3,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bedroom ?,[yes],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
4,"[mary, moved, to, the, bathroom, ., sandra, jo...",is daniel in the bedroom ?,[yes],"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
...,...,...,...,...,...,...,...
9995,"[daniel, went, to, the, office, ., sandra, jou...",is sandra in the kitchen ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, kitchen, ?]","['sandra', 'kitchen']",['no']
9996,"[daniel, went, to, the, office, ., sandra, jou...",is daniel in the bedroom ?,[yes],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
9997,"[daniel, went, to, the, office, ., sandra, jou...",is sandra in the hallway ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
9998,"[daniel, went, to, the, office, ., sandra, jou...",is mary in the kitchen ?,[no],"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, mary, in, the, kitchen, ?]","['mary', 'kitchen']",['no']


In [ ]:
df3.drop('Context', axis=1, inplace=True)
df3.drop('Answer', axis=1, inplace=True)




In [ ]:
df3


,Question,Context_final,Quesiton,Question_final,Answer_final
0,is sandra in the hallway ?,"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
1,is daniel in the bathroom ?,"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bathroom, ?]","['daniel', 'bathroom']",['no']
2,is daniel in the office ?,"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, office, ?]","['daniel', 'office']",['no']
3,is daniel in the bedroom ?,"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
4,is daniel in the bedroom ?,"['mary', 'move', 'bathroom', 'sandra', 'journe...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
...,...,...,...,...,...
9995,is sandra in the kitchen ?,"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, kitchen, ?]","['sandra', 'kitchen']",['no']
9996,is daniel in the bedroom ?,"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, daniel, in, the, bedroom, ?]","['daniel', 'bedroom']",['yes']
9997,is sandra in the hallway ?,"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, sandra, in, the, hallway, ?]","['sandra', 'hallway']",['no']
9998,is mary in the kitchen ?,"['daniel', 'go', 'office', 'sandra', 'journeye...","[is, mary, in, the, kitchen, ?]","['mary', 'kitchen']",['no']


In [ ]:
# Assuming df_test has columns 'Context_final' and 'Question_final'
df_test = df3

# Tokenize the words in the 'Context_final' and 'Question_final' columns using the same tokenizer
X_sequences_context_test = tokenizer.texts_to_sequences(df_test['Context_final'])
X_sequences_question_test = tokenizer.texts_to_sequences(df_test['Question_final'])

# Pad sequences to make them of equal length
X_padded_context_test = pad_sequences(X_sequences_context_test)
X_padded_question_test = pad_sequences(X_sequences_question_test)

# Concatenate the padded sequences
X_combined_padded_test = np.concatenate((X_padded_context_test, X_padded_question_test), axis=1)

# Reshape X to have 3D shape (number of samples, time steps, number of features)
X_combined_padded_test = X_combined_padded_test.reshape(X_combined_padded_test.shape[0], 1, X_combined_padded_test.shape[1])

# Make predictions
predictions = model.predict(X_combined_padded_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Decode class labels to get the original context values
decoded_predictions = encoder_context.inverse_transform(predicted_labels)

# Now, 'decoded_predictions' contains the predicted context values for the test data


313/313 [==============================] - 1s 2ms/step


In [ ]:
y2=df3['Answer_final']

In [ ]:

Encoder = LabelEncoder()
Test_y = Encoder.fit_transform(y2)


In [ ]:
Test_y

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
decoded_predictions

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import f1_score


# Calculate F1 score
f1 = f1_score(Test_y, decoded_predictions, average='weighted')

print(f'F1 Score: {f1:.4f}')

accuracy = np.mean(Test_y == decoded_predictions)

print(f'Accuracy: {accuracy * 100:.2f}%')

F1 Score: 0.8184
Accuracy: 81.85%


In [ ]:
import pandas as pd
import numpy as np


# Create a DataFrame
df_results = pd.DataFrame({
    'Actual_Labels': Test_y,
    'Predicted_Labels': decoded_predictions
})

# Replace 1 with 'Yes' and 0 with 'No'
df_results.replace({1: 'Yes', 0: 'No'}, inplace=True)

# Display the resulting DataFrame
print(df_results)


     Actual_Labels Predicted_Labels
0               No               No
1               No               No
2               No               No
3              Yes              Yes
4              Yes              Yes
...            ...              ...
9995            No               No
9996           Yes               No
9997            No               No
9998            No               No
9999           Yes               No

[10000 rows x 2 columns]
